# Cryptographie — Travail pratique 2

**Auteurs**
- Jeddi Youssef
- Themlaoui Houssem

**Lien repo Github**  : 
https://github.com/ThemlaouiHou/travail-pratique-1-




## Exercice 1 — Accès GitLab (SSH)

**-----------------------------------Réponses de Themlaoui Houssem ----------------------------------**


**Clé utilisée pour GitLab HE-Arc (gitlab-etu.ing.he-arc.ch)**
- Commande (auth OK sur le serveur): `ssh -vT git@gitlab-etu.ing.he-arc.ch`
- Algorithme (commande): `ssh -vT git@gitlab-etu.ing.he-arc.ch` → ED25519
- Longueur de clé (commande): `ssh-keygen -lf $env:USERPROFILE\.ssh\id_ed25519_p3.pub` → 256 bits
- Clé publique (commande): `Get-Content $env:USERPROFILE\.ssh\id_ed25519_p3.pub` → `ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAID3E5YpFilFU4fX9PLMwvX3/Ykg6io/KgY9lN92rLpKL houssem.themlaoui@he-arc.ch`

**Sécurité (attaques classiques)**
- Oui — ED25519 (clé 256 bits, sécurité ≈ 128 bits) est aujourd’hui considéré comme sûr face aux attaques classiques. Avec des ordinateurs conventionnels et les méthodes connues, il n’existe pas de moyen pratique pour retrouver la clé privée ni pour forger des signatures valides. Donc ,pas de régénération nécessaire.

**Sécurité (ordinateur quantique)**
- Non, ED25519 ne résiste pas à un adversaire quantique (Shor).Parce qu’ED25519 repose sur la difficulté du logarithme discret sur courbe elliptique. Un ordinateur quantique exécutant l’algorithme de Shor résout ce problème en temps polynomial, donc il peut récupérer la  clé privée à partir de la clé publique.


**-----------------------------------Réponses de Jeddi Youssef ----------------------------------**

**a. Algorithme et longueur de clé**
- Commande pour générer une nouvelle clé Ed25519: `ssh-keygen -t ed25519 -C "youssef.jeddi@he-arc.ch"`. Aucune passphrase n'a été choisie, le champ a été laissé vide.
- Affichage de la clé publique:
  - sous Windows: `Get-Content $env:USERPROFILE\.ssh\id_ed25519.pub`
  - sous Linux `cat ~/.ssh/id_ed25519.pub`
  - Résultat: `ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIN866GaiTiVHr+oRepDf1FVItsvuicrxw9Bggt5PDcS5 youssef.jeddi@he-arc.ch`
- Vérifier l'algorithme de chiffrement et la taille de la clé:
  - sous Windows: `ssh-keygen -lf $env:USERPROFILE\.ssh\id_ed25519.pub`
  - sous Linux: `ssh-keygen -lf ~/.ssh/id_ed25519.pub`
  - Résultat: `256 SHA256:QVbCUheLlXfhcMQ0kuLDKkqNGZE8P9+CPZ6oU0546XY youssef.jeddi@he-arc.ch (ED25519)`
    - Taille de la clé en bits: `256`
    - Algorithme utilisé: `(ED25519)`

Contrairement à RSA, ED25519 a une taille fixe et correspond à un niveau de sécurité d’environ 128 bits contre les attaques classiques.
On ne peut pas générer une clé ED25519 "plus grande" ou "plus petite".

**b. Solution secure vs attaques classique?**

- Comme vu en classe, le niveau de sécurité de 128 bits est aujourd’hui considéré comme suffisant contre les attaques classiques (brute-force). Ce niveau a été illustré à l’aide d’AES-128 durant le cours même si l'algorithme n'est pas le même dans notre cas.

- Dans notre situation, un attaquant chercherait soit à retrouver la clé privée, soit à forger une signature valide. Cela reviendrait à résoudre un logarithme discret sur une courbe elliptique, problème considéré comme impraticable avec des ordinateurs classiques et les algorithmes connus à ce jour.

- Une régénération de clé n’est nécessaire qu’en cas de compromission de la clé privée (par exemple suite à une erreur humaine) ou si l’algorithme devenait obsolète.

**c. Solution secure vs ordinateur quantique?**

- Dans le cas hypothétique ou un ordinateur quantique assez puissant existait, certains problèmes mathématique deviendraient plus faciles. Notamment la résolution de la **factorisation (RSA)** ou le **logarithme discret(Diffie-Hellman, ECC)** serait possible grâce à **l'algorithme de Shor**. Donc, il peut casser **Ed25519** utilisé par Gitlab.
- Donc **NON**, si cet ordinateur quantique universel existait, un attaquant pourrait retrouver ma clé privée à partir de la clé publique et s'authentifier à ma place.
- À l’heure actuelle, les algorithmes post-quantiques ne sont pas encore déployés pour l’authentification SSH sur GitLab. Ed25519 reste la solution recommandée.


## Exercice 2 — Devoir 7, exercice 6 (hachage)

On considère la fonction de hachage:
`H_{pub}(x) = (1103515245 x + 12345) mod 327680`


### Code (verbatim depuis `devoir7_ex6/solution.py`)


In [1]:
# Hash value, collisions, and linear-congruence solution.
import math

A = 1103515245
B = 12345
m = 327680

x = 47572947294858218452
h = (A * x + B) % m
hash_bits = math.log2(m)

h_values = [118290, 215350, 311620, 118975, 54830]

# Brute force: find two inputs per target hash.
collisions = {h: [] for h in h_values}
candidate = 0
while any(len(v) < 2 for v in collisions.values()):
    hv = (A * candidate + B) % m
    if hv in collisions and len(collisions[hv]) < 2:
        collisions[hv].append(candidate)
    candidate += 1

# Non-brute-force: solve linear congruence.
d = math.gcd(A, m)
A1 = A // d
m1 = m // d

def egcd(a, b):
    if b == 0:
        return (a, 1, 0)
    g, x1, y1 = egcd(b, a % b)
    return (g, y1, x1 - (a // b) * y1)

g, inv, _ = egcd(A1, m1)
if g != 1:
    raise ValueError("No modular inverse")
inv %= m1

# All solutions: x = x0 + k * m1.
all_solutions = {}
for hv in h_values:
    rhs = (hv - B) % m
    if rhs % d != 0:
        all_solutions[hv] = []
        continue
    x0 = (inv * (rhs // d)) % m1
    all_solutions[hv] = [x0 + k * m1 for k in range(d)]

# Results
print("hash(x) =", h)
print("hash_bits =", hash_bits)
print("bruteforce_collisions =", collisions)
print("linear_solutions =", all_solutions)


hash(x) = 283005
hash_bits = 18.32192809488736
bruteforce_collisions = {118290: [31133, 96669], 215350: [6865, 72401], 311620: [21079, 86615], 118975: [22238, 87774], 54830: [24745, 90281]}
linear_solutions = {118290: [31133, 96669, 162205, 227741, 293277], 215350: [6865, 72401, 137937, 203473, 269009], 311620: [21079, 86615, 152151, 217687, 283223], 118975: [22238, 87774, 153310, 218846, 284382], 54830: [24745, 90281, 155817, 221353, 286889]}


## Réponse écrite devoir 7

**Procédé (code)**
- Calcul de H_pub(x) et de la taille en bits (log2 m).
- Recherche de collisions par brute force sur la plage des x.
- Résolution analytique des collisions via pgdc(A, m) et inverse modulaire.


In [2]:
A = 1103515245
B = 12345
m = 327680
x = 47572947294858218452

h = (A * x + B) % m
h


283005

**Réponse 1**: \(H_{pub}(x) = 283005\)


In [3]:
import math
# Output size in bits for modulus m.
math.log2(327680)


18.32192809488736

**Réponse 1 (suite)**: taille ≈ 18.32 bits (≈ 18 bits).


In [ ]:
A = 1103515245
B = 12345
m = 327680
h_values = [118290, 215350, 311620, 118975, 54830]


results = {h: [] for h in h_values}

x = 0
while any(len(v) < 2 for v in results.values()):
    hv = (A * x + B) % m
    if hv in results and len(results[hv]) < 2:
        results[hv].append(x)
    x += 1

results


{118290: [31133, 96669],
 215350: [6865, 72401],
 311620: [21079, 86615],
 118975: [22238, 87774],
 54830: [24745, 90281]}

**Réponse 2 (collisions)**
- h1 = 118290: x = 31133 et 96669
- h2 = 215350: x = 6865 et 72401
- h3 = 311620: x = 21079 et 86615
- h4 = 118975: x = 22238 et 87774
- h5 = 54830:  x = 24745 et 90281


**Réponse 3 : Pourquoi la fonction est cassée + collisions sans brute force ?**

H_pub(x) = (A x + B) mod m est linéaire.  
Avec A = 1103515245, B = 12345, m = 327680,  
d = gcd(A, m) = 5.                              Chaque hash a 5 préimages (si (h - B) multiple de d).  

On résout A x ≡ (h - B) (mod m) :  
- A' = A / d, m' = m / d  
- x0 = A'^{-1} * ((h - B)/d) (mod m')  
- Solutions : x = x0 + k * m' pour k = 0..d-1  

Cela redonne les collisions ci-dessus sans brute force.


## Exercice 3 — Devoir 8, exercice 6 (Diffie-Hellman)

Données:
- `n = 1005658541636276696854926736111523240565378642222118817358603616124640001`
- `g = 2`
- `A = 694345273912301015795665084471934281997943951751938128184262171033972161`
- `B = 45219809272265904064518788699770061112569516384310833106022960617800893`


### Code (verbatim depuis `devoir8_ex6/solution.py`)


In [ ]:
# Pohlig-Hellman + CRT for the DH parameters.
import math
from collections import Counter

n = 1005658541636276696854926736111523240565378642222118817358603616124640001
g = 2
A = 694345273912301015795665084471934281997943951751938128184262171033972161
B = 45219809272265904064518788699770061112569516384310833106022960617800893

# Factorization from FactorDB 
p = 845722348837208875924502082373440001
q = 1189111938473620218791973405751200001

# Order of g=2 in Z_p* and Z_q* 
ord_p = 84572234883720887592450208237344000
ord_q = 28826956084208975001017537109120

# Prime-power factorization of each order.
factors_p = Counter({3: 9, 2: 8, 31: 6, 19: 5, 11: 4, 5: 3, 7: 3, 23: 3})
factors_q = Counter({2: 7, 3: 6, 23: 5, 17: 5, 29: 5, 7: 3, 31: 2, 5: 1})


def dlog_prime_power(g_base, h_base, mod, order, prime, exp):
    # Lift discrete log over a prime power.
    n_i = prime ** exp
    m = order // n_i
    g1 = pow(g_base, m, mod)
    h1 = pow(h_base, m, mod)
    gk = pow(g1, prime ** (exp - 1), mod) # order prime
    x = 0
    for k in range(exp):
        c = (h1 * pow(g1, -x, mod)) % mod
        c = pow(c, prime ** (exp - 1 - k), mod)
        # Find the next digit in base "prime".
        d = None
        cur = 1
        for i in range(prime):
            if cur == c:
                d = i
                break
            cur = (cur * gk) % mod
        if d is None:
            raise ValueError("Discrete log failed for prime power")
        x += d * (prime ** k)
    return x


def pohlig_hellman(g_base, h_base, mod, order, factors):
    # Solve discrete log per prime power, then combine with CRT.
    congruences = []
    for prime, exp in factors.items():
        x_i = dlog_prime_power(g_base, h_base, mod, order, prime, exp)
        congruences.append((x_i, prime ** exp))
    # CRT, pairwise coprime moduli
    x = 0
    N = 1
    for _, ni in congruences:
        N *= ni
    for ai, ni in congruences:
        Mi = N // ni
        inv = pow(Mi, -1, ni)
        x = (x + ai * Mi * inv) % N
    return x


def crt_general(a1, n1, a2, n2):
    # CRT for possibly non-coprime moduli.
    g = math.gcd(n1, n2)
    if (a1 - a2) % g != 0:
        raise ValueError("No CRT solution")
    lcm = n1 // g * n2
    m1 = n1 // g
    m2 = n2 // g
    t = ((a2 - a1) // g) * pow(m1, -1, m2) % m2
    x = (a1 + n1 * t) % lcm
    return x, lcm


# Solve discrete logs mod p and q, then combine.
ap = pohlig_hellman(g % p, A % p, p, ord_p, factors_p)
aq = pohlig_hellman(g % q, A % q, q, ord_q, factors_q)

a, lcm_ord = crt_general(ap, ord_p, aq, ord_q)

# Shared key.
k = pow(B, a, n)

# Results
print("ap =", ap)
print("aq =", aq)
print("a =", a)
print("lcm_ord =", lcm_ord)
print("check A:", pow(g, a, n) == A)
print("shared key k =", k)


## Réponse écrite devoir 8

**Procédé (code)**
- Factorisation de n, puis calcul des ordres de g modulo p et q.
- Discrets logs via Pohlig–Hellman sur p et q.
- Combinaison par CRT pour obtenir a, puis calcul de k = B^a mod n.


**Factorisation de n** (via FactorDB):
- Factorisation de n obtenue via FactorDB (base publique), calculs cryptographiques réalisés localement.
- n = p * q
- \(p = 845722348837208875924502082373440001\)
- \(q = 1189111938473620218791973405751200001\)


**Ordres de g=2**
- ord_p = 84572234883720887592450208237344000
- ord_q = 28826956084208975001017537109120



**Pohlig–Hellman + CRT**
- a ≡ 62625926665364995110065259106721457 (mod ord_p)
- a ≡ 23795998967845128119911284363057 (mod ord_q)

Donc:
- \(a = 697755647111918244911954894837420708027739461921457\)


**Clé partagée**
- \(k = B^a mod n\)
- \(k = 922254424716744089879181468690343626534229719971081549439971098559365319\)


## Usage d'un LLM 

Un LLM (mistral) a été utilisé pour clarifier la démarche, rappeler des notions
(Pohlig‑Hellman, CRT) et relire la formulation. Les idées mathématiques, les calculs,la validation des résultats et la rédaction finale ont été réalisés par l’équipe. La majorité du travail est manuelle (70–80%). L’usage du LLM a concerné uniquement l’exercice 3 (Devoir 8, ex6).                                                                                   
                                                                                                        